<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Converting-to-qiime2-artifacts" data-toc-modified-id="Converting-to-qiime2-artifacts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Converting to qiime2 artifacts</a></span><ul class="toc-item"><li><span><a href="#OTU-table" data-toc-modified-id="OTU-table-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>OTU table</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Taxonomy" data-toc-modified-id="Taxonomy-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Taxonomy</a></span></li><li><span><a href="#Phylogeny" data-toc-modified-id="Phylogeny-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Phylogeny</a></span></li></ul></li><li><span><a href="#Calculating-diversity" data-toc-modified-id="Calculating-diversity-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculating diversity</a></span></li><li><span><a href="#Converting-to-R-objects" data-toc-modified-id="Converting-to-R-objects-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Converting to R objects</a></span><ul class="toc-item"><li><span><a href="#alpha-diversity" data-toc-modified-id="alpha-diversity-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>alpha-diversity</a></span><ul class="toc-item"><li><span><a href="#Compiling-into-a-list" data-toc-modified-id="Compiling-into-a-list-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Compiling into a list</a></span></li></ul></li><li><span><a href="#beta-diversity" data-toc-modified-id="beta-diversity-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>beta diversity</a></span><ul class="toc-item"><li><span><a href="#Compiling-into-a-list" data-toc-modified-id="Compiling-into-a-list-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Compiling into a list</a></span></li></ul></li></ul></li><li><span><a href="#Final-files" data-toc-modified-id="Final-files-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Final files</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* calculate alpha and beta diversity via qiime2

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac/'

# 16S-arch phyloseq object; rarefied
#physeq_arc_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/physeq_arch_arc-bac_r250.RDS'
# 16S-bac phyloseq object; rarefied
physeq_bac_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/physeq_bac_arc-bac_r5k.RDS'

# misc
tmp_dir = file.path(work_dir, 'qiime2_files')
conda_env = 'qiime2-2019.10'

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(phyloseq)
library(ape)
library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
df.dims()
make_dir(work_dir)
make_dir(tmp_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac/ 
Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files 


# Converting to qiime2 artifacts

In [4]:
# loading phyloseq object
physeq = readRDS(physeq_bac_file)
physeq 

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 30361 taxa and 140 samples ]
sample_data() Sample Data:       [ 140 samples by 171 sample variables ]
tax_table()   Taxonomy Table:    [ 30361 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 30361 tips and 29814 internal nodes ]

In [5]:
# aggregating by genus
physeq = tax_glom(physeq, 'Genus')
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 817 taxa and 140 samples ]
sample_data() Sample Data:       [ 140 samples by 171 sample variables ]
tax_table()   Taxonomy Table:    [ 817 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 817 tips and 816 internal nodes ]

## OTU table

In [6]:
# converting OTU table
tmp = physeq %>% phyloseq2df(otu_table)
tmp = cbind(tmp %>% dplyr::select(OTU),
            tmp %>% dplyr::select(-OTU))
tmp

OTU,X101_Horse,X102_Kulan,X109_Red_Fox,X111_West_European_Hedgehog,X116_Common_Kestrel,X122_Wild_Boar,X123_Alpine_Chamois,X127_Alpine_Marmot,X128_Alpine_Marmot,⋯,F45_Red_Deer,F47_Red_Deer,F48_Red_Deer,F53_Mouflon,F66_Wild_Boar,F68_Red_Deer,F69_Red_Deer,F70_Red_Deer,F80_Red_Deer,F90_Domestic_Dog
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1781517ce246703d6b462685778525d2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2c8d504055d7d4ed6b09da3b03b3eca9,1,2,0,0,0,1,0,284,1,⋯,0,0,0,0,1,0,0,0,1,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
d0208c53dfa85aa1dcbd2e39ad1dd38b,419,187,7,72,0,341,155,172,191,⋯,450,476,186,388,213,305,369,316,309,256
976a36c8eb0079f5a1aef9b2b8f5c820,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [7]:
# OTU table value summary
rowSums(tmp[,2:ncol(tmp)]) %>% summary %>% print
colSums(tmp[,2:ncol(tmp)]) %>% summary %>% print

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0     4.0    19.0   833.8   185.0 98090.0 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   4245    4778    4931    4866    4999    5000 


In [8]:
# writing out table
otu_table_file = file.path(tmp_dir, 'physeq_otu-table_genus.tsv')
write.table(tmp, otu_table_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', otu_table_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu-table_genus.tsv 


In [9]:
# converting otu to biom
otu_biom_file = file.path(tmp_dir, 'physeq_otu-table_genus.biom')
cmd = sprintf('biom convert -i %s -o %s --to-hdf5', otu_table_file, otu_biom_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "biom convert -i /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu-table_genus.tsv -o /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu-table_genus.biom --to-hdf5"


NULL

In [10]:
# converting biom to qiime2 artifact
otu_art_file = file.path(tmp_dir, 'physeq_otu_genus.qza')
cmd = sprintf('qiime tools import --type FeatureTable[Frequency] --input-path %s --output-path %s', 
              otu_biom_file, otu_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type FeatureTable[Frequency] --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu-table_genus.biom --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu_genus.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu-table_genus.biom as BIOMV210DirFmt to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu_genus.qza

NULL

## Metadata

In [11]:
# converting sample metadata
tmp = physeq %>% phyloseq2df(sample_data) %>%
    mutate(X.SampleID = gsub('\\.', '_', X.SampleID)) %>%
    dplyr::select(-sample_name)
tmp = cbind(tmp %>% dplyr::select(X.SampleID),
            tmp %>% dplyr::select(-X.SampleID))
colnames(tmp)[1] = '#SampleID'
tmp$host_class = tmp$class
tmp 

#SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description,Sample,host_class
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>
X101_Horse,ACGACGTCTTAG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0030,101,101,Hauspferd,Horse,⋯,344,160,160,NA,2,20,101,NA,X101_Horse,Mammalia
X102_Kulan,ACGAGTGCTATC,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0031,102,102,Khulan,Kulan,⋯,345,161,161,NA,2,20,102,NA,X102_Kulan,Mammalia
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
F80_Red_Deer,ACTACGTGTGGT,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0046,F80,F80,Rothirsch,Red.Deer,⋯,116,14,14,NA,7,20,F80,NA,F80_Red_Deer,Mammalia
F90_Domestic_Dog,ACTCGATTCGAT,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0051,F90,F90,Hund,Domestic.Dog,⋯,118,16,16,NA,7,20,F90,NA,F90_Domestic_Dog,Mammalia


In [12]:
# writing out table
sample_data_file = file.path(tmp_dir, 'physeq_sample-data.tsv')
write.table(tmp, sample_data_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', sample_data_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_sample-data.tsv 


## Taxonomy

In [13]:
# converting taxonomy
tmp = physeq %>% phyloseq2df(tax_table)
tmp = cbind(tmp %>% dplyr::select(OTU),
            tmp %>% dplyr::select(-OTU))
tmp = tmp %>%
    unite(taxonomy, Domain, Phylum, Class, Order, Family, Genus, sep=';') %>%
    mutate(taxonomy = gsub(';+$','', taxonomy))
tmp

OTU,taxonomy,Species
<chr>,<chr>,<chr>
1781517ce246703d6b462685778525d2,Bacteria;Candidate division SR1;unclassified;unclassified;unclassified;unclassified,NA
2c8d504055d7d4ed6b09da3b03b3eca9,Bacteria;Proteobacteria;unclassified;unclassified;unclassified;unclassified,NA
⋮,⋮,⋮
d0208c53dfa85aa1dcbd2e39ad1dd38b,Bacteria;Firmicutes;Clostridia;Clostridiales;Lachnospiraceae;unclassified,NA
976a36c8eb0079f5a1aef9b2b8f5c820,Bacteria;Firmicutes;Clostridia;Clostridiales;Lachnospiraceae;Johnsonella,NA


In [14]:
# writing out table
tax_table_file = file.path(tmp_dir, 'physeq_tax-table_genus.tsv')
write.table(tmp, tax_table_file, sep='\t', quote=FALSE, row.names=FALSE, col.names=FALSE)
cat('File written:', tax_table_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_tax-table_genus.tsv 


In [15]:
# converting tsv to artifact
tax_art_file = file.path(tmp_dir, 'physeq_tax_genus.qza')
cmd = sprintf('qiime tools import --type FeatureData[Taxonomy] --input-format HeaderlessTSVTaxonomyFormat --input-path %s --output-path %s', 
              tax_table_file, tax_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type FeatureData[Taxonomy] --input-format HeaderlessTSVTaxonomyFormat --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_tax-table_genus.tsv --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_tax_genus.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_tax-table_genus.tsv as HeaderlessTSVTaxonomyFormat to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_tax_genus.qza

NULL

## Phylogeny

In [16]:
tree_file = file.path(tmp_dir, 'physeq_16S_genus.nwk')
write.tree(physeq %>% phy_tree, tree_file)
cat('File written:', tree_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16S_genus.nwk 


In [17]:
# convert to artifact
tree_art_file = file.path(tmp_dir, 'physeq_16Stree_genus.qza')
cmd = sprintf('qiime tools import --type Phylogeny[Rooted] --input-path %s --output-path %s', 
              tree_file, tree_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type Phylogeny[Rooted] --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16S_genus.nwk --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16Stree_genus.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16S_genus.nwk as NewickDirectoryFormat to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16Stree_genus.qza

NULL

# Calculating diversity

In [40]:
# core-metrics: phylogenetic diversity function

## output 
core_metrics_phylo_dir = file.path(tmp_dir, 'core_metrics_phylo_genus')
unlink(core_metrics_phylo_dir, recursive=TRUE)
log_file = file.path(tmp_dir, 'core-metrics-phylo_genus.log')


## command
cmd = "qiime diversity core-metrics-phylogenetic \\
  --i-phylogeny %s \\
  --i-table %s \\
  --m-metadata-file %s \\
  --output-dir %s \\
  --p-sampling-depth %s \\
  2> %s 1>&2"

cmd = sprintf(cmd,
              tree_art_file,
              otu_art_file, 
              sample_data_file, 
              core_metrics_phylo_dir,
              4000, log_file)

## run
cat(cmd)
bash_job(cmd, conda_env)

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_16Stree_genus.qza \
  --i-table /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_otu_genus.qza \
  --m-metadata-file /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/physeq_sample-data.tsv \
  --output-dir /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus \
  --p-sampling-depth 4000 \
  2> /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core-metrics-phylo_genus.log 1>&2

NULL

# Converting to R objects

## alpha-diversity

In [41]:
# phylo vectors
files = list.files(core_metrics_phylo_dir, pattern='*_vector.qza', full.names=TRUE)
files

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/evenness_vector.qza"     
[2] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/faith_pd_vector.qza"     
[3] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/observed_otus_vector.qza"
[4] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/shannon_vector.qza"

In [42]:
# phylo vectors
cmd = "qiime tools export --output-path %s --input-path %s"

for(f in files){
    # export qza to tsv
    cmd_tmp = sprintf(cmd, core_metrics_phylo_dir, f)
    cat(cmd_tmp, '\n')
    bash_job(cmd_tmp, conda_env)    
          
    # convert tsv to data.frame RDS
    exported_file = file.path(core_metrics_phylo_dir, 'alpha-diversity.tsv')
    df = read.delim(exported_file, sep='\t', row.names=1)
    final_file = gsub('_vector.+', '_genus.RDS', f)
    final_file = file.path(work_dir, basename(final_file))
    saveRDS(df, final_file)
    cat('\nFile written:', final_file, '\n\n')
}

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/evenness_vector.qza 
Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/evenness_vector.qza as AlphaDiversityDirectoryFormat to directory /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus
File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//evenness_genus.RDS 

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data

### Compiling into a list

In [43]:
alpha_div = list(
   'evenness' = 'evenness_genus.RDS',
   'observed' = 'observed_otus_genus.RDS',
   'shannon' = 'shannon_genus.RDS',
   'faith_pd' = 'faith_pd_genus.RDS'
)

alpha_div_rds = list()
for(n in names(alpha_div)){
    x = readRDS(file.path(work_dir, alpha_div[[n]]))
    alpha_div_rds[[n]] = x
}

alpha_div_rds_file = file.path(work_dir, 'alpha_div_genus.RDS')
saveRDS(alpha_div_rds, alpha_div_rds_file)
cat('File written:', alpha_div_rds_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//alpha_div_genus.RDS 


In [44]:
# checking dimensions
alpha_div_rds %>%
    lapply(function(x) dim(as.matrix(x)))

$evenness
[1] 140   1

$observed
[1] 140   1

$shannon
[1] 140   1

$faith_pd
[1] 140   1

## beta diversity

In [45]:
# phylo distance matrices
files = list.files(core_metrics_phylo_dir, pattern='*matrix.qza', full.names=TRUE)
files

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/bray_curtis_distance_matrix.qza"       
[2] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/jaccard_distance_matrix.qza"           
[3] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/unweighted_unifrac_distance_matrix.qza"
[4] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/weighted_unifrac_distance_matrix.qza"

In [46]:
# phylo distance matrices
cmd = "qiime tools export --output-path %s --input-path %s"

for(f in files){
    # export qza to tsv
    cmd_tmp = sprintf(cmd, core_metrics_phylo_dir, f)
    cat(cmd_tmp, '\n')
    bash_job(cmd_tmp, conda_env)    
          
    # convert tsv to data.frame RDS
    exported_file = file.path(core_metrics_phylo_dir, 'distance-matrix.tsv')
    d = read.delim(exported_file, sep='\t', row.names=1) %>% as.dist
    final_file = gsub('_distance_matrix.+', '_genus.RDS', f)
    final_file = file.path(work_dir, basename(final_file))
    saveRDS(d, final_file)
    cat('\nFile written:', final_file, '\n\n')
}

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/bray_curtis_distance_matrix.qza 
Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//qiime2_files/core_metrics_phylo_genus
File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//bray_curtis_genus.RDS 

qiime tools export --output-path /ebio/abt3_proje

### Compiling into a list

In [47]:
beta_div = list(
   'bray_curtis' = 'bray_curtis_genus.RDS',
   'jaccard' = 'jaccard_genus.RDS',
   'unweighted_unifrac' = 'unweighted_unifrac_genus.RDS',
   'weighted_unifrac' = 'weighted_unifrac_genus.RDS'
)

beta_div_rds = list()
for(n in names(beta_div)){
    x = readRDS(file.path(work_dir, beta_div[[n]]))
    beta_div_rds[[n]] = x
}

beta_div_rds_file = file.path(work_dir, 'beta_div_genus.RDS')
saveRDS(beta_div_rds, beta_div_rds_file)
cat('File written:', beta_div_rds_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//beta_div_genus.RDS 


In [48]:
# checking dimensions
beta_div_rds %>% 
    lapply(function(x) dim(as.matrix(x)))

$bray_curtis
[1] 140 140

$jaccard
[1] 140 140

$unweighted_unifrac
[1] 140 140

$weighted_unifrac
[1] 140 140

# Final files

In [49]:
print(alpha_div_rds_file)
print(beta_div_rds_file)

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//alpha_div_genus.RDS"
[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/arch-bac/diversity-qiime2_bac//beta_div_genus.RDS"


# sessionInfo

In [50]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ape_5.4           phyloseq_1.30.0   ggplot2_3.3.1    
[5] tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6        lattice_0.20-41     Biostrings_2.54.0  
 [4] digest_0.6.25       foreach_1.5.0       IRdisplay_0.7